# Step 3 - Consolidating Data

This notebook is designed to consolidate ORES article evaluation data, and U.S. population data, into a single file. It takes in the output of the Step 2 notebook, which consists of thousands of JSON files that each have an individual ORES evaluation as well as metadata on the article that was evaluated. It also takes in  the static files: an Excel file containing population data for U.S. states, named "NST-EST2022-POP.xslx" (this file can be downloaded from the website for the [U.S. Census Bureau](https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html)), a CSV file containing a list of Wikipedia city articles and what state they correspond to, called "us\_cities\_by\_state\_SEPT.2023.csv" (the same file used as input for the Step 1 notebook, although its code didn't use the state column data), and an Excel file containing information about which regional division a state belongs to, called "US States by Region - US Census Bureau.xlsx".

In [1]:
#This cell defines constants and imports the notebook will use
import os, json
import pandas as pd

ores_files_path = "raw_api_data/ores/"
cities_by_state_path = "input/us_cities_by_state_SEPT.2023.csv"
states_by_region_path = "input/US States by Region - US Census Bureau.xlsx"
excel_path = 'input/NST-EST2022-POP.xlsx'
output_path = "refined_data/wp_scored_city_articles_by_state.csv"

In [2]:
#This cell takes in all three static tabular data files, and joins them
#together into one Pandas Dataframe

#Importing list of Wikipedia city articles and their states
cities_by_state_data = pd.read_csv(cities_by_state_path)

#Importing State population Data
raw_excel = pd.read_excel(excel_path, index_col=0, header = 3)
raw_excel_length, raw_excel_width = raw_excel.shape
excel_data = raw_excel.iloc[0:(raw_excel_length - 7), ]
excel_data = excel_data.rename(columns={'Unnamed: 1': 'April 1, 2020 Estimates Base'})
excel_data['Geographic Area'] = excel_data.index.values
labels = list(range(excel_data.shape[0]))
excel_data.index = labels


#Fixing the state name columns in both previous DataFrames so they match, and
#then joining them by state name
def fix_state_names(state_string):
    newstring = state_string
    if state_string[0] == '.':
        newstring = state_string[1:]
    
    newstring = newstring.replace("_(U.S._state)", "")
    newstring = newstring.replace("_", " ")
    
    return newstring   
excel_data['Geographic Area'] = [fix_state_names(x) for x in list(excel_data['Geographic Area'])]
cities_by_state_data['state'] = [fix_state_names(x) for x in list(cities_by_state_data['state'])]
joined_static_data = cities_by_state_data.join(excel_data.set_index('Geographic Area'), on='state')
joined_static_data = joined_static_data[joined_static_data['April 1, 2020 Estimates Base'].notna()]

#Grabbing the state grouping data, and then joining it with the previous joined DataFrame
states_by_region = pd.read_excel(states_by_region_path, header = 0)
states_by_region['is_state'] = states_by_region['STATE'].notna()
states_by_region = states_by_region.fillna(method='ffill')
states_by_region = states_by_region[states_by_region['is_state']]
joined_static_data = joined_static_data.join(states_by_region.set_index('STATE'), on='state')
joined_static_data = joined_static_data[joined_static_data['REGION'].notna()]
joined_static_data

,state,page_title,url,"April 1, 2020 Estimates Base",2020,2021,2022,REGION,DIVISION,is_state
0,Alabama,"Abbeville, Alabama","https://en.wikipedia.org/wiki/Abbeville,_Alabama",5024356.0,5031362.0,5049846.0,5074296.0,South,East South Central,True
1,Alabama,"Adamsville, Alabama","https://en.wikipedia.org/wiki/Adamsville,_Alabama",5024356.0,5031362.0,5049846.0,5074296.0,South,East South Central,True
2,Alabama,"Addison, Alabama","https://en.wikipedia.org/wiki/Addison,_Alabama",5024356.0,5031362.0,5049846.0,5074296.0,South,East South Central,True
3,Alabama,"Akron, Alabama","https://en.wikipedia.org/wiki/Akron,_Alabama",5024356.0,5031362.0,5049846.0,5074296.0,South,East South Central,True
4,Alabama,"Alabaster, Alabama","https://en.wikipedia.org/wiki/Alabaster,_Alabama",5024356.0,5031362.0,5049846.0,5074296.0,South,East South Central,True
...,...,...,...,...,...,...,...,...,...,...
22152,Wyoming,"Wamsutter, Wyoming","https://en.wikipedia.org/wiki/Wamsutter,_Wyoming",576837.0,577605.0,579483.0,581381.0,West,Mountain,True
22153,Wyoming,"Wheatland, Wyoming","https://en.wikipedia.org/wiki/Wheatland,_Wyoming",576837.0,577605.0,579483.0,581381.0,West,Mountain,True
22154,Wyoming,"Worland, Wyoming","https://en.wikipedia.org/wiki/Worland,_Wyoming",576837.0,577605.0,579483.0,581381.0,West,Mountain,True
22155,Wyoming,"Wright, Wyoming","https://en.wikipedia.org/wiki/Wright,_Wyoming",576837.0,577605.0,579483.0,581381.0,West,Mountain,True


In [3]:
#This cell loops every individual JSON file generated by the Step 2 Notebook,
#and combines them all into one Dataframe, keeping only the three fields
#needed for the final data file. 

file_list = os.listdir(ores_files_path)
#file_list = file_list[1:5]
ores_dataframe = []
for filename in file_list:
    json_path = ores_files_path + filename
    file = open(json_path)
    example_json = json.load(file)
    file.close()
    request_info = example_json['request_info']
    lastrevid = request_info['lastrevid']
    prediction = example_json['scores'][str(lastrevid)]['articlequality']['score']['prediction']
    new_list_object = {
        'article_title': request_info['page_title'],
        'lastrevid' : lastrevid,
        'article_quality': prediction
    }
    #print(request_info['page_title'])
    ores_dataframe.append(new_list_object)
ores_dataframe = pd.DataFrame(ores_dataframe)
print(ores_dataframe)

                             article_title   lastrevid article_quality
0                   Princes Lakes, Indiana  1019560872            Stub
1      Montrose Charter Township, Michigan   101989287            Stub
2                        Holcomb, New York  1024625267            Stub
3                     Moyie Springs, Idaho  1032509578               C
4                          Kilauea, Hawaii  1036838051            Stub
...                                    ...         ...             ...
21514                         Omao, Hawaii   965363482            Stub
21515                    Chicago, Illinois   967239270            Stub
21516                      Waikane, Hawaii   982615222            Stub
21517      Kailua, Honolulu County, Hawaii   985095782            Stub
21518               Matoaka, West Virginia   999751661               C

[21519 rows x 3 columns]


In [4]:
#This cell joins the ORES data with the static data and filters out all non-city article rows
#and unneeded columns, and then exports the much cleaner resuling DataFrame as a CSV

ores_static_join = ores_dataframe.join(joined_static_data.set_index('page_title'), on='article_title')
ores_static_join = ores_static_join[ores_static_join['REGION'].notna()]
ores_static_join = ores_static_join.drop_duplicates() 
ores_static_join = ores_static_join[ores_static_join['url'] != "https://en.wikipedia.org/wiki/Population"] 
ores_static_join = ores_static_join[ores_static_join['url'] != "https://en.wikipedia.org/wiki/Square_mile"] 
ores_static_join = ores_static_join[ores_static_join['url'] != "https://en.wikipedia.org/wiki/County_(United_States)"] 
ores_static_join = ores_static_join[ores_static_join['url'] != "https://en.wikipedia.org/wiki/2010_United_States_census"] 
ores_static_join = ores_static_join[ores_static_join['url'] != "https://en.wikipedia.org/wiki/2020_United_States_census"] 

wp_scored_city_articles_by_state = pd.DataFrame({
    "state": ores_static_join['state'],
    "regional_division": ores_static_join['DIVISION'],
    "population": ores_static_join[2022],
    "article_title": ores_static_join['article_title'],
    "revision_id": ores_static_join['lastrevid'],
    "article_quality": ores_static_join['article_quality']
})

wp_scored_city_articles_by_state.to_csv(output_path, index=False)